In [2]:
import pretty_midi
import math
import os
import csv
from pathlib import Path

%load_ext autoreload
%autoreload 2

NUM_SONGS = 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def difficulty(midi_path):
    pm = pretty_midi.PrettyMIDI(midi_path)

    beats = pm.get_beats()

    i = 0

    for inst in pm.instruments:
        for note in inst.notes:
            i += 1

    notes_per_beat = i / len(beats)

    diff = round(notes_per_beat/2, 2)

    return diff

In [5]:
with open("manifest.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["audio_filepath", "midi_filepath", "difficulty"])

for i in range(NUM_SONGS):
    try:
        diff = difficulty(f"midi_done/{i}.mid")
        
        print(diff)
        with open("manifest.csv", "a", newline="") as f:
            writer = csv.writer(f)
            midi_filepath = fr"midi_done\{i}.mid"
            audio_filepath = fr"song\{i}.mp3"
            writer.writerow([audio_filepath, midi_filepath, diff])
    except:
        pass

3.59
1.33
2.86
3.4
3.85
3.88
0.81
1.98
1.44
1.83
3.23
3.33
2.66
2.54
3.45
2.45
3.31
4.49
2.71
3.28
2.72
3.7
4.26
3.69
2.94
2.73
2.59
3.88
3.4
2.93
3.36
1.05
1.01
2.01
2.66
2.89
1.24
2.48
